In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-52361079-3dd1-307a-4402-e81341128dc7)


In [ ]:
%%capture install_log 

!pip install transformers datasets evaluate optuna

In [ ]:
import torch
import optuna
import evaluate
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorForLanguageModeling,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gngpostalsrvc/BERiT_2000_custom_architecture_150_epochs_2')

def preprocess(examples):
  
    encoding = tokenizer(examples['Text'], max_length=128, truncation=True, padding=True)
    encoding['labels'] = [[stage] for stage in examples['Stage']]

    return encoding

raw_data = load_dataset('gngpostalsrvc/COHeN')

tokenized_data = raw_data.map(preprocess, batched=True, remove_columns=raw_data['train'].column_names)
tokenized_data.set_format("pt", columns=["input_ids", "attention_mask", "labels"], output_all_columns=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1197 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1197 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/9574 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/gngpostalsrvc___parquet/gngpostalsrvc--COHeN-97096b619f4d4787/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/1197 [00:00<?, ? examples/s]

Map:   0%|          | 0/1197 [00:00<?, ? examples/s]

Map:   0%|          | 0/9574 [00:00<?, ? examples/s]

In [5]:
def compute_metrics(eval_preds):
    metrics = evaluate.load('accuracy')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)


def objective(trial):
  model = AutoModelForSequenceClassification.from_pretrained('gngpostalsrvc/BERiT', num_labels=4)
  batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128])
  args = TrainingArguments(output_dir="opt-test", 
                         evaluation_strategy="epoch",
                         learning_rate=trial.suggest_float('learning_rate', low=4e-5, high=.01),
                         weight_decay=trial.suggest_float('weight_decay', low=4e-5, high=.01),
                         num_train_epochs=3,
                         per_device_train_batch_size=batch_size, 
                         per_device_eval_batch_size=batch_size, 
                         seed=42,
                         disable_tqdm=True
                        )
  
  trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
  )

  result = trainer.train()

  return result.training_loss

study = optuna.create_study(study_name='hp-search-COHeN', direction='minimize')
study.optimize(func=objective, n_trials=20)

[I 2023-03-05 16:49:39,626] A new study created in memory with name: hp-search-COHeN


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gngpostalsrvc--BERiT/snapshots/5799f2933c5845e55984f628ee81b985752c3897/config.json
Model config RobertaConfig {
  "_name_or_path": "gngpostalsrvc/BERiT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.5,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 128,
  "model_type": "roberta",
  "num_attention_heads": 4,
  "num_hidden_layers": 1,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--gngpostalsrvc--BERiT/snapshots/5799f2933c5845e55984f628ee81b985752c3897/pytorch_model.bin
Some weights of the model checkpoint at gngpostalsrvc/BERiT were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from t

{'loss': 1.3993, 'learning_rate': 0.005234378203240644, 'epoch': 0.83}


***** Running Evaluation *****
  Num examples = 1197
  Batch size = 16


...

{'loss': 1.1347, 'learning_rate': 0.001438975144983106, 'epoch': 1.67}


***** Running Evaluation *****
  Num examples = 1197
  Batch size = 32


{'eval_loss': 0.8189311027526855, 'eval_accuracy': 0.6666666666666666, 'eval_runtime': 2.4248, 'eval_samples_per_second': 493.649, 'eval_steps_per_second': 15.671, 'epoch': 2.0}


***** Running Evaluation *****
  Num examples = 1197
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


[I 2023-03-05 17:04:12,729] Trial 19 finished with value: 1.0276928032769097 and parameters: {'batch_size': 32, 'learning_rate': 0.0032376940762119883, 'weight_decay': 0.005012474188757684}. Best is trial 16 with value: 1.0100221082899306.


{'eval_loss': 0.7353609204292297, 'eval_accuracy': 0.720969089390142, 'eval_runtime': 2.4489, 'eval_samples_per_second': 488.799, 'eval_steps_per_second': 15.517, 'epoch': 3.0}
{'train_runtime': 40.7823, 'train_samples_per_second': 704.277, 'train_steps_per_second': 22.068, 'train_loss': 1.0276928032769097, 'epoch': 3.0}


{'batch_size': 32, 'learning_rate': 0.0027431492469971175, 'weight_decay': 0.004900150335195089}